In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from scipy.linalg import sqrtm
from skimage.metrics import structural_similarity
import lpips
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc



class ImageMetricsComparisonSimple:
    def __init__(self, fake_dir, inv_fake_dir, real_dir, inv_real_dir, 
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.fake_dir = fake_dir
        self.inv_fake_dir = inv_fake_dir
        self.real_dir = real_dir
        self.inv_real_dir = inv_real_dir
        self.device = device
        self.target_size = (1024, 1024)
        
        
        self.loss_fn_alex = lpips.LPIPS(net='alex').to(device)

    def calculate_psnr(self, img1, img2):
        if isinstance(img1, Image.Image):
            img1 = img1.resize(self.target_size, Image.Resampling.LANCZOS)
            img1 = np.array(img1)
        if isinstance(img2, Image.Image):
            img2 = img2.resize(self.target_size, Image.Resampling.LANCZOS)
            img2 = np.array(img2)
        
        img1 = img1.astype(np.float64)
        img2 = img2.astype(np.float64)
        
        mse = np.mean((img1 - img2) ** 2)
        if mse == 0:
            return float('inf')
        
        max_pixel = 255.0
        psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
        return psnr

    def calculate_ssim(self, img1, img2):
        if isinstance(img1, Image.Image):
            img1 = img1.resize(self.target_size, Image.Resampling.LANCZOS)
            img1 = np.array(img1)
        if isinstance(img2, Image.Image):
            img2 = img2.resize(self.target_size, Image.Resampling.LANCZOS)
            img2 = np.array(img2)
            
        if img1.dtype != np.uint8:
            img1 = (img1.clip(0, 255) + 0.5).astype(np.uint8)
        if img2.dtype != np.uint8:
            img2 = (img2.clip(0, 255) + 0.5).astype(np.uint8)
            
        if len(img1.shape) == 3:
            img1 = np.mean(img1, axis=2).astype(np.uint8)
        if len(img2.shape) == 3:
            img2 = np.mean(img2, axis=2).astype(np.uint8)
            
        return structural_similarity(img1, img2, data_range=255)

    def calculate_lpips(self, img1, img2):
        if not isinstance(img1, Image.Image):
            img1 = Image.fromarray(img1)
        if not isinstance(img2, Image.Image):
            img2 = Image.fromarray(img2)
            
        img1 = img1.resize(self.target_size, Image.Resampling.LANCZOS)
        img2 = img2.resize(self.target_size, Image.Resampling.LANCZOS)
        
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
        
        img1_tensor = transform(img1).unsqueeze(0).to(self.device)
        img2_tensor = transform(img2).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            lpips_value = self.loss_fn_alex(img1_tensor, img2_tensor)
        return lpips_value.item()

    def calculate_metrics_for_pair(self, dir1, dir2, desc):
        files = sorted([f for f in os.listdir(dir1) if f.endswith('.png')])
        metrics = {
            'psnr': [], 'ssim': [], 'lpips': [], 'filenames': []
        }
        
        for filename in tqdm(files, desc=desc):
            path1 = os.path.join(dir1, filename)
            path2 = os.path.join(dir2, filename)
            
            if not os.path.exists(path2):
                continue
            
            try:
                img1 = Image.open(path1).convert('RGB')
                img2 = Image.open(path2).convert('RGB')
                
                psnr = self.calculate_psnr(img1, img2)
                ssim = self.calculate_ssim(img1, img2)
                lpips_val = self.calculate_lpips(img1, img2)
                
                if all(x is not None and not np.isnan(x) for x in [psnr, ssim, lpips_val]):
                    metrics['psnr'].append(psnr)
                    metrics['ssim'].append(ssim)
                    metrics['lpips'].append(lpips_val)
                    metrics['filenames'].append(filename)
                
            except Exception:
                continue
        
        return metrics

    def plot_comparative_metrics(self, fake_metrics, real_metrics):
        if len(fake_metrics['filenames']) == 0 or len(real_metrics['filenames']) == 0:
            return

    def plot_comparative_metrics(self, fake_metrics, real_metrics):
        if len(fake_metrics['filenames']) == 0 or len(real_metrics['filenames']) == 0:
            print("Error: No data available to plot metrics. Please check if the input directories contain matching image pairs.")
            return

        fig = plt.figure(figsize=(15, 12))
        gs = plt.GridSpec(3, 2)  # Added an extra row for ROC curve
        
        metrics_list = [
            ('PSNR (Higher is better)', 'psnr', gs[0, 0], True),
            ('SSIM (Higher is better)', 'ssim', gs[0, 1], True),
            ('LPIPS (Lower is better)', 'lpips', gs[1, 0], False)
        ]
        
        # Plot histograms
        for title, metric_name, pos, higher_better in metrics_list:
            ax = fig.add_subplot(pos)
            fake_values = np.array(fake_metrics[metric_name])
            real_values = np.array(real_metrics[metric_name])
            
            fake_values = fake_values[~np.isinf(fake_values)]
            real_values = real_values[~np.isinf(real_values)]
            
            if len(fake_values) > 0 and len(real_values) > 0:
                ax.hist(fake_values, bins=30, alpha=0.5, label='Fake vs Inv_Fake', color='blue')
                ax.hist(real_values, bins=30, alpha=0.5, label='Real vs Inv_Real', color='red')
                ax.set_title(title)
                ax.legend()
                ax.grid(True, alpha=0.3)
                
                fake_stats = f'Fake μ={np.mean(fake_values):.3f}, σ={np.std(fake_values):.3f}'
                real_stats = f'Real μ={np.mean(real_values):.3f}, σ={np.std(real_values):.3f}'
                ax.text(0.02, 0.98, f'{fake_stats}\n{real_stats}',
                    transform=ax.transAxes, va='top', fontsize=8)
            else:
                ax.text(0.5, 0.5, 'No data available', 
                    ha='center', va='center', transform=ax.transAxes)

        # Plot ROC curves
        ax_roc = fig.add_subplot(gs[1, 1])
        metrics_for_roc = ['psnr', 'ssim', 'lpips']
        
        if len(fake_metrics['filenames']) > 0 and len(real_metrics['filenames']) > 0:
            for metric_name in metrics_for_roc:
                # For LPIPS, lower is better so we need to invert the scores
                scores = np.concatenate([fake_metrics[metric_name], real_metrics[metric_name]])
                if metric_name == 'lpips':
                    scores = -scores  # Invert LPIPS scores since lower is better
                
                # Create labels: 1 for fake, 0 for real
                labels = np.concatenate([np.ones(len(fake_metrics[metric_name])), 
                                    np.zeros(len(real_metrics[metric_name]))])
                
                # Remove inf values
                valid_idx = ~np.isinf(scores)
                scores = scores[valid_idx]
                labels = labels[valid_idx]
                
                if len(scores) > 0 and len(np.unique(labels)) > 1:
                    fpr, tpr, _ = roc_curve(labels, scores)
                    roc_auc = auc(fpr, tpr)
                    ax_roc.plot(fpr, tpr, label=f'{metric_name.upper()} (AUC = {roc_auc:.2f})')
            
            ax_roc.plot([0, 1], [0, 1], 'k--')
            ax_roc.set_title('ROC Curves')
            ax_roc.set_xlabel('False Positive Rate')
            ax_roc.set_ylabel('True Positive Rate')
            ax_roc.legend()
            ax_roc.grid(True, alpha=0.3)
        else:
            ax_roc.text(0.5, 0.5, 'No data available for ROC curves', 
                    ha='center', va='center')

        # Plot correlation matrix
        ax_corr = fig.add_subplot(gs[2, :])  # Make correlation matrix span both columns
        metrics_data = []
        metric_labels = ['PSNR', 'SSIM', 'LPIPS']
        for metric in ['psnr', 'ssim', 'lpips']:
            combined_metric = np.concatenate([fake_metrics[metric], real_metrics[metric]])
            combined_metric = combined_metric[~np.isinf(combined_metric)]
            if len(combined_metric) > 0:
                metrics_data.append(combined_metric)
        
        if len(metrics_data) > 0:
            metrics_data = np.array(metrics_data)
            corr_matrix = np.corrcoef(metrics_data)
            
            im = ax_corr.imshow(corr_matrix, cmap='coolwarm', aspect='auto')
            ax_corr.set_title('Metrics Correlation')
            
            ax_corr.set_xticks(range(len(metric_labels)))
            ax_corr.set_yticks(range(len(metric_labels)))
            ax_corr.set_xticklabels(metric_labels)
            ax_corr.set_yticklabels(metric_labels)
            
            for i in range(len(metric_labels)):
                for j in range(len(metric_labels)):
                    ax_corr.text(j, i, f'{corr_matrix[i, j]:.2f}', 
                            ha='center', va='center')
            
            plt.colorbar(im, ax=ax_corr)
        else:
            ax_corr.text(0.5, 0.5, 'No data available', 
                    ha='center', va='center', transform=ax_corr.transAxes)
        
        plt.tight_layout()
        plt.savefig('metrics_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        self.save_numerical_results(fake_metrics, real_metrics)

    def save_numerical_results(self, fake_metrics, real_metrics):
        with open('metrics_results.txt', 'w') as f:
            f.write("=== Numerical Results ===\n\n")
            
            for metric in ['psnr', 'ssim', 'lpips']:
                f.write(f"\n{metric.upper()} Statistics:\n")
                
                fake_values = np.array(fake_metrics[metric])
                real_values = np.array(real_metrics[metric])
                
                fake_values = fake_values[~np.isinf(fake_values)]
                real_values = real_values[~np.isinf(real_values)]
                
                f.write(f"Fake vs Inv_Fake:\n")
                f.write(f"  Mean: {np.mean(fake_values):.4f}\n")
                f.write(f"  Std:  {np.std(fake_values):.4f}\n")
                f.write(f"  Min:  {np.min(fake_values):.4f}\n")
                f.write(f"  Max:  {np.max(fake_values):.4f}\n")
                
                f.write(f"Real vs Inv_Real:\n")
                f.write(f"  Mean: {np.mean(real_values):.4f}\n")
                f.write(f"  Std:  {np.std(real_values):.4f}\n")
                f.write(f"  Min:  {np.min(real_values):.4f}\n")
                f.write(f"  Max:  {np.max(real_values):.4f}\n")


def main():
    base_dir = '/shared/shashmi/inversion/inversion/SD3'
    fake_dir = os.path.join(base_dir, 'fake')
    inv_fake_dir = os.path.join(base_dir, 'inv_fake')
    real_dir = os.path.join(base_dir, 'real')
    inv_real_dir = os.path.join(base_dir, 'inv_real')
    
    metrics_calc = ImageMetricsComparisonSimple(
        fake_dir, inv_fake_dir, real_dir, inv_real_dir
    )
    
    fake_metrics = metrics_calc.calculate_metrics_for_pair(fake_dir, inv_fake_dir, "Fake vs Inv_Fake")
    real_metrics = metrics_calc.calculate_metrics_for_pair(real_dir, inv_real_dir, "Real vs Inv_Real")
    metrics_calc.plot_comparative_metrics(fake_metrics, real_metrics)

if __name__ == "__main__":
    try:
        torch.multiprocessing.set_start_method('spawn')
    except RuntimeError:
        pass
    main()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/shared/shashmi/conda_envs/inverter/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/shared/shashmi/conda_envs/inverter/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /shared/shashmi/conda_envs/inverter/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


/shared/shashmi/conda_envs/inverter/lib/python3.10/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path